In [1]:
!pip install PyPDF2 faiss-cpu sentence_transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=aae62ad31d47ac66976ba7601a43f548eda5bc3feba682e1c165bb3f48cd3f46
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [12]:
from PyPDF2 import PdfReader
from faiss import IndexFlatL2
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import numpy as np

In [14]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")


In [ ]:
def process_pdf(pdf_path):
    if pdf_path is not None:
        pdf_reader = PdfReader(pdf_path)

        text = ""

        for page in pdf_reader.pages:
            text += page.extract_text() or ""

        return text

In [ ]:
def embed_text(text):
    return model.encode(text)

In [ ]:
def build_vector_store(texts):

    vectors = [embed_text(text) for text in texts]

    vectors_np = np.array(vectors).astype('float32')

    index = IndexFlatL2(vectors_np.shape[1])
    index.add(vectors_np)
    return index

In [ ]:
def query_vector_store(query, index, texts):
    query_embedding = embed_text(query)
    distances, indices = index.search(query_embedding.reshape(1, -1), k=5)
    return [texts[i] for i in indices[0]]

In [ ]:
def answer_question(question, passage):
    answer = qa_model(question=question, context=passage)
    return answer["answer"]


In [ ]:
def chat(pdf_path):

    text = process_pdf(pdf_path)
    sentences = text.strip().split("\n")
    vector_store = build_vector_store(sentences)

    # Chat loop
    while True:
        user_query = input("Ask your question: ")
        if user_query == "quit":
            break

        relevant_sentences = query_vector_store(user_query, vector_store, sentences)
        for sentence in relevant_sentences:
            answer = answer_question(user_query, sentence)
            print(f"\nPossible Answer: {answer}")

        print("\n")

In [13]:
# Run the chatbot
chat("222.pdf")

Ask your question: What is CNN?

Possible Answer: a CNN

Possible Answer: CNN

Possible Answer: models

Possible Answer: expressive power

Possible Answer: models


Ask your question: Rnn

Possible Answer: 16 to 128 neu rons

Possible Answer: N.

Possible Answer: networks

Possible Answer: LeNet

Possible Answer: 1


Ask your question: LeNet

Possible Answer: LeNet

Possible Answer: Advantages of LeNet

Possible Answer: 21 List the advantages of LeNet. How many layers

Possible Answer: convolutional and pooling layers

Possible Answer: AlexNet


Ask your question: What the PDF about

Possible Answer: Characteristics

Possible Answer: preservatio n

Possible Answer: Here's how it can be applied

Possible Answer: large-scale and diverse set of labelled  images

Possible Answer: ca tegorize images into different classes or labels


Ask your question: Title

Possible Answer: 1

Possible Answer: 1

Possible Answer: introduction

Possible Answer: backpropagation

Possible Answer: Characteris

KeyboardInterrupt: ignored